In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:

cmap = plt.get_cmap('inferno')     #making an instance of color map with color code inferno

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/home/oldsport/Downloads/genres/{g}'):
        songname = f'/home/oldsport/Downloads/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
        
        
# Mono or monophonic describes a system where all the audio signals are mixed together 
# and routed through a single audio channel. Mono systems can have multiple loudspeakers,
# and even multiple widely separated loudspeakers.

#Duration in Seconds

#NFFT: The number of data points used in each block for the FFT
#The benefit of a longer FFT is that you will have more frequency resolution.
#The larger the number the longer the computation takes

#Fs, Fc noverlap are taken as default values

# scale is log scale with db

<Figure size 720x720 with 0 Axes>

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [6]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/home/oldsport/Downloads/genres/{g}'):
        songname = f'/home/oldsport/Downloads/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        rmse= librosa.feature.rmse(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

# librosa.core.load(path, sr=22050, mono=True, offset=0.0, duration=None, dtype=<class 'numpy.float32'>, res_type='kaiser_best')

In [8]:
data = pd.read_csv('data.csv')
data.shape

(1000, 28)

In [9]:
data = data.drop(['filename'],axis=1)

In [10]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)     #Encode labels with value between 0 and n_classes-1

In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

#Standardize features by removing the mean and scaling to unit variance

# The standard score of a sample x is calculated as:

# z = (x - u) / s
# where u is the mean of the training samples or zero if with_mean=False,
# and s is the standard deviation of the training samples or one if with_std=False.



In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:

X_train[10]

array([ 2.47725996, -0.26080111,  0.63502638,  0.18013298,  0.47925077,
        1.31867374,  0.6886862 , -0.56361769, -0.60648238,  2.01556139,
       -0.86650641,  0.9380281 , -1.34662769,  1.61586014, -0.85582562,
        1.81460617, -0.89227961,  1.55987946, -1.41456037,  0.80455888,
       -1.28868676,  1.23026279, -1.43210825,  0.9989935 , -0.83494794,
        1.05900279])

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 2.1969 - acc: 0.2075
Epoch 2/20
800/800 [==============================] - 0s 33us/step - loss: 1.8423 - acc: 0.3625
Epoch 3/20
800/800 [==============================] - 0s 27us/step - loss: 1.5787 - acc: 0.4150
Epoch 4/20
800/800 [==============================] - 0s 25us/step - loss: 1.3767 - acc: 0.5262
Epoch 5/20
800/800 [==============================] - 0s 43us/step - loss: 1.2405 - acc: 0.5787
Epoch 6/20
800/800 [==============================] - 0s 35us/step - loss: 1.1258 - acc: 0.6412
Epoch 7/20
800/800 [==============================] - 0s 23us/step - loss: 1.0431 - acc: 0.6475
Epoch 8/20
800/800 [==============================] - 0s 15us/step - loss: 0.9802 - acc: 0.6687
Epoch 9/20
800/800 [==============================] - 0s 15us/step - loss: 0.9213 - acc: 0.7075
Epoch 10/20
800/800 [==============================] - 0s 14us/step - loss: 0.8710 - acc:

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 173us/step


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.65


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [19]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [20]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 432us/step - loss: 2.3103 - acc: 0.0917 - val_loss: 2.1814 - val_acc: 0.2350
Epoch 2/30
600/600 [==============================] - 0s 50us/step - loss: 2.1177 - acc: 0.3283 - val_loss: 2.0707 - val_acc: 0.2550
Epoch 3/30
600/600 [==============================] - 0s 53us/step - loss: 1.9658 - acc: 0.3583 - val_loss: 1.9620 - val_acc: 0.3050
Epoch 4/30
600/600 [==============================] - 0s 47us/step - loss: 1.8217 - acc: 0.4083 - val_loss: 1.8760 - val_acc: 0.3400
Epoch 5/30
600/600 [==============================] - 0s 49us/step - loss: 1.7040 - acc: 0.4267 - val_loss: 1.7814 - val_acc: 0.3650
Epoch 6/30
600/600 [==============================] - 0s 48us/step - loss: 1.5828 - acc: 0.4417 - val_loss: 1.6746 - val_acc: 0.4150
Epoch 7/30
600/600 [==============================] - 0s 50us/step - loss: 1.4711 - acc: 0.5083 - val_loss: 1.6061 - val_acc: 0.3850
Epoch 8/30
600/600 [==

In [21]:
results

[1.2775928354263306, 0.595]

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

1.0000001

In [25]:
np.argmax(predictions[0])

7